In [1]:
import numpy as np
import pandas as pd
import pickle


from matplotlib import pyplot as plt
%matplotlib inline 

from utils.train_conformal import train_conformal_forecaster

In [ ]:
# Dataset size
# n_train_samples = 1000
# n_calibration_samples = 1000
# n_test_samples = 500

# Time series parameters
seq_len = 10
horizon = 5
noise_profile = None
periodicity = 2
amplitude = 1
dynamic_sequence_lengths=False

# LSTM parameters
epochs=4000
batch_size=100
embedding_size=20
coverage=0.9
lr=0.01


In [ ]:
def run_synthetic_experiments(exp_mode, params, coverage, seq_len,
                              n_train_seq, n_test_seq, retrain=False):
    
    baselines           = ["BJRNN", "QRNN", "DPRNN"]
    exp_res_dict        = dict({"BJRNN": [], "QRNN": [], "DPRNN": []})
    baseline_results    = dict({"BJRNN": dict({"CI_length": None, "Errors": None,  "Coverages": None, }), 
                                "QRNN": dict({"CI_length": None, "Errors": None,  "Coverages": None, }), 
                                "DPRNN": dict({"CI_length": None, "Errors": None,  "Coverages": None, })})
    
    if exp_mode=="2":
        
        n_samples_          = [10, 100, 1000]
        noise_var_          = 0.2
        
        if retrain:
    
            for n_sample_ in n_samples_:

                exp_results     = collect_synthetic_results(noise_var_, params, coverage=coverage, seq_len=seq_len,
                                                            n_train_seq=n_train_seq, n_test_seq=n_test_seq)

                exp_res_dict["BJRNN"].append(exp_results["BJRNN"])
                exp_res_dict["QRNN"].append(exp_results["QRNN"])
                exp_res_dict["DPRNN"].append(exp_results["DPRNN"])
        else:
            
            infile              = open('saved_models/Experiment_2_result','rb')
            exp_res_dict        = pickle.load(infile)
            
    elif exp_mode=="1":
            
        noise_vars          = [0.1 * k for k in range(9)]    
        
        if retrain:
            
            for noise_var in noise_vars:

                exp_results   = collect_synthetic_results(noise_var, params, coverage=coverage, seq_len=seq_len,
                                                          n_train_seq=n_samples, n_test_seq=n_test)

                exp_res_dict["BJRNN"].append(exp_results["BJRNN"])
                exp_res_dict["QRNN"].append(exp_results["QRNN"])
                exp_res_dict["DPRNN"].append(exp_results["DPRNN"])
        
        else:
            
            infile              = open('saved_models/Experiment_1_result','rb')
            exp_res_dict        = pickle.load(infile)
        
        
    for baseline in baselines:
        
        baseline_results[baseline]["CI_length"] = [exp_res_dict[baseline][k][0]["CI length"] for k in range(len(exp_res_dict[baseline]))]
        baseline_results[baseline]["Errors"]    = [np.mean(np.concatenate(exp_res_dict[baseline][k][0]["Errors"])) for k in range(len(exp_res_dict[baseline]))]
        baseline_results[baseline]["Coverages"] = BJRNN_coverages = [exp_res_dict[baseline][k][0]["Coverage"] for k in range(len(exp_res_dict[baseline]))]
    
    return baseline_results

## Data processing

In [3]:
df = pd.read_csv('data/energy_data.csv')

In [4]:
df

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,60,30,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,45.566667,...,17.033333,45.5300,6.600000,733.5,92.000000,7.000000,63.000000,5.300000,13.275433,13.275433
1,60,30,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,45.992500,...,17.066667,45.5600,6.483333,733.6,92.000000,6.666667,59.166667,5.200000,18.606195,18.606195
2,50,30,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,45.890000,...,17.000000,45.5000,6.366667,733.7,92.000000,6.333333,55.333333,5.100000,28.642668,28.642668
3,50,40,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,45.723333,...,17.000000,45.4000,6.250000,733.8,92.000000,6.000000,51.500000,5.000000,45.410389,45.410389
4,60,40,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,45.530000,...,17.000000,45.4000,6.133333,733.9,92.000000,5.666667,47.666667,4.900000,10.084097,10.084097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,100,0,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,45.590000,...,23.200000,46.7900,22.733333,755.2,55.666667,3.333333,23.666667,13.333333,43.096812,43.096812
19731,90,0,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,45.590000,...,23.200000,46.7900,22.600000,755.2,56.000000,3.500000,24.500000,13.300000,49.282940,49.282940
19732,270,10,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,45.730000,...,23.200000,46.7900,22.466667,755.2,56.333333,3.666667,25.333333,13.266667,29.199117,29.199117
19733,420,10,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.700000,45.790000,...,23.200000,46.8175,22.333333,755.2,56.666667,3.833333,26.166667,13.233333,6.322784,6.322784
